# ЛР2 Формирование отчётов в Apache Spark

Инициализация и загрузка необходимых библиотек и других элементов

In [ ]:
!pip install pyspark
import pyspark
import os

from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

#Чтение данных из CSV и XML файлов в Apache Spark, а также преобразование DataFrame в список строк.
sc = SparkSession.builder.appName("Lab2").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0").getOrCreate()
languages = sc.read.csv("/content/programming-languages.csv")
languages = [str(x[0]) for x in languages.collect()]
posts = sc.read.format('xml').options(rowTag='row').load("/content/posts_sample.xml")

Задание:\
Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: RDD API, Dataset API, SQL API.

In [ ]:
from datetime import datetime
#Фильтрация объектов по году создания
def get_year(x, year):
  return x._CreationDate >= datetime(year=year, month=1, day=1) and x._CreationDate <= datetime(year=year, month=12, day=31)

def text_lower(x):
  tag = next((language for language in languages if "<" + language.lower()+ ">" in x._Tags.lower()), None)
  return (x._Id, tag) if tag is not None else None

dataframe = {}

def create_df(year):
      #Проверка на наличие тэга и года
      #Преобрзование в нижний регистр, создание кортежа, если найден язык, убираем пустые значения
      #Группировка по значению в кортеже и агреггация
      #Сортировка по убыванию и формирование dataframe
  dataframe[year] = posts.rdd.filter(lambda x: x._Tags is not None and get_year(x, year))\
      .map(text_lower).filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1]).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2)\
      .sortBy(lambda x: x[1], ascending=False).toDF()
  dataframe[year] = dataframe[year].select(col("_1").alias("Lang"),col("_2").alias(f"Count_in_{year}")).limit(10)
  dataframe[year].show()

In [ ]:
#Формируем отчет за 2010-2020 года
for y in range(2010, 2020):
  create_df(y)

#Сохраняем в формате
for year in dataframe.keys():
    dataframe[year].write.format("parquet").save(f"/content/info_from_{year}")

+-----------+-------------+
|       Lang|Count_in_2010|
+-----------+-------------+
| JavaScript|           20|
|       Java|           15|
|        PHP|           15|
|          C|            8|
|Objective-C|            7|
|       Ruby|            5|
|     Python|            5|
|       Bash|            2|
|     Delphi|            2|
|       Perl|            2|
+-----------+-------------+

+-----------+-------------+
|       Lang|Count_in_2011|
+-----------+-------------+
|       Java|           32|
|        PHP|           25|
| JavaScript|           18|
|          C|           10|
|     Python|           10|
|Objective-C|            9|
|       Perl|            6|
|       Ruby|            5|
| ColdFusion|            3|
|       Bash|            2|
+-----------+-------------+

+-----------+-------------+
|       Lang|Count_in_2012|
+-----------+-------------+
| JavaScript|           49|
|        PHP|           48|
|       Java|           47|
|     Python|           23|
|Objective-C|     

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/info_from_2010 already exists. Set mode as "overwrite" to overwrite the existing path.